# Stripped down Pinwheel annotator test area

In [4]:
import os 
import numpy as np
import xarray as xr
from datetime import date, timedelta, datetime
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from matplotlib.image import imread
import geocat.viz.util as gvutil
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

from collections import deque
import matplotlib.patches as patches
from pathlib import Path
import json

In [5]:
class EyedropperAnnotator:
    def __init__(self):
        self.brown_events = deque()
        
    def add_events(self, centroids, sizes, current_time):
        """Add new events with sizes at current time"""
        for (lat, lon), size in zip(centroids, sizes):
            self.brown_events.append((lon, lat, current_time, size))
    
    def save_state(self, filepath):
        """Overwrite current state (for restart)"""
        def convert_event(event):
            return tuple(float(x) if isinstance(x, (np.floating, np.integer)) else x for x in event)
        
        state = {'brown_events': [convert_event(e) for e in self.brown_events]}
        with open(filepath, 'w') as f:
            json.dump(state, f)

    def append_history(self, filepath):
        """Append all events to growing file"""
        def convert_event(event):
            return tuple(float(x) if isinstance(x, (np.floating, np.integer)) else x for x in event)
        
        snapshot = {'brown_events': [convert_event(e) for e in self.brown_events]}
        with open(filepath, 'a') as f:
            f.write(json.dumps(snapshot) + '\n')

    @staticmethod
    def load_state(filepath):
        """Load state for restart, or create new if file doesn't exist"""
        if not Path(filepath).exists():
            return EyedropperAnnotator()
        with open(filepath, 'r') as f:
            state = json.load(f)
        annotator = EyedropperAnnotator()
        annotator.brown_events = deque(state['brown_events'])
        return annotator


    def plot_one_expanding_ring(self, ax, current_time, facecolor='orange'):
        """Plot expanding annuli with size-dependent opacity"""
        import matplotlib.patches as patches
        import cartopy.crs as ccrs
        
        to_remove = []
        for i, (lon, lat, birth_time, size) in enumerate(list(self.brown_events)):
            age = current_time - birth_time
            if age < 0:
                continue
                
            outer_r = age * 52 * 60 * 30 / 111111.1
            area = np.pi * (outer_r**2)
            alpha = min(0.8, size / max(area, 1))
            
            if alpha > 0.01:
                if outer_r > 0:
                    annulus = patches.Annulus((lon, lat), outer_r, width=outer_r/2, 
                                          facecolor=facecolor, alpha=alpha, 
                                          edgecolor='none', transform=ccrs.PlateCarree())
                    ax.add_patch(annulus)
            else:
                to_remove.append(i)
        
        for i in reversed(to_remove):
            del self.brown_events[i]

    def plot_expanding_ring_coriolis(self, ax, current_time, facecolor='orange', 
                                     spoke_color='red', n_spokes=12):
        """
        Plot expanding annuli with Coriolis-curved spokes.
        
        The spokes show counterclockwise rotation due to Coriolis force:
        - Each spoke curves from inner_r to outer_r
        - Older (larger radius) parts lag behind in azimuth
        - Mean azimuth rotates counterclockwise with age
        
        Args:
            ax: matplotlib axis
            current_time: current frame time
            facecolor: color of annulus
            spoke_color: color of Coriolis spokes
            n_spokes: number of spokes (default 12)
        """
    
        to_remove = []
        for i, (lon, lat, birth_time, size) in enumerate(list(self.brown_events)):
            age = current_time - birth_time
            if age < 0:
                continue
                
            outer_r = age    * 52 * 60 * 30 / 111111.1  # degrees
            inner_r = (age-1)* 52 * 60 * 30 / 111111.1
            width =         1* 52 * 60 * 30 / 111111.1
            area = np.pi * (outer_r**2 - inner_r**2)
            alpha = min(0.8, size / max(area, 1))
            
            if alpha > 0.01:
                if outer_r > 0:
                    # Draw base annulus
                    annulus = patches.Annulus((lon, lat), outer_r, width=width, 
                                          facecolor=facecolor, alpha=alpha, 
                                          edgecolor='none', transform=ccrs.PlateCarree())
                    ax.add_patch(annulus)
                    
                    # Draw Coriolis spokes
                    # Coriolis rotation: f = 2*Omega*sin(lat), integrated over time
                    # Approximate angular displacement: delta_theta ∝ age^2 * sin(lat)
                    lat_rad = np.radians(lat)
                    coriolis_factor = np.sin(lat_rad)  # Coriolis strength
                    
                    # Mean counterclockwise rotation increases with age
                    mean_rotation = -coriolis_factor * age * 50  # degrees, tunable
                    
                    # Differential rotation: inner parts rotate more than outer
                    # (because they've experienced Coriolis longer)
                    rotation_gradient = coriolis_factor * age * 20  # degrees, tunable
                    
                    for spoke_idx in range(n_spokes):
                        # Base azimuth for this spoke
                        base_azimuth = spoke_idx * 360 / n_spokes
                        
                        # Create curved spoke from inner_r to outer_r
                        n_points = 20
                        radii = np.linspace(inner_r, outer_r, n_points)
                        
                        # Azimuth varies along spoke due to Coriolis
                        # Inner radius rotates more (older trajectory)
                        azimuths = base_azimuth + mean_rotation + \
                                                rotation_gradient * (1 - radii/outer_r)
                        
                        # Convert to lon/lat offsets
                        x_offsets = radii * np.cos(np.radians(azimuths))
                        y_offsets = radii * np.sin(np.radians(azimuths))
                        
                        # Create curved path
                        spoke_lons = lon + x_offsets
                        spoke_lats = lat + y_offsets
                        
                        # Plot as curved line
                        ax.plot(spoke_lons, spoke_lats, color=spoke_color, 
                               alpha=alpha, linewidth=1.5, 
                               transform=ccrs.PlateCarree())
                        
            else:
                to_remove.append(i)
        
        for i in reversed(to_remove):
            del self.brown_events[i]

    def plot_expanding_ring_coriolis_pinwheel(self, ax, current_time, facecolor='orange', 
                                     spoke_color='white', n_spokes=12):
        """
        Plot expanding annuli with Coriolis-curved spokes.
        
        The spokes show rotation proportional to Coriolis force.        
        Args:
            ax: matplotlib axis
            current_time: current frame time
            facecolor: color of annulus
            spoke_color: color of Coriolis spokes
            n_spokes: number of spokes (default 12)
        """
        
        to_remove = []
        for i, (lon, lat, birth_time, size) in enumerate(list(self.brown_events)):
            age = current_time - birth_time  # seconds 
            if age < 0:
                continue
                
            outer_r = age    * 52 * 60 * 30 / 111111.1  # degrees
            inner_r = (age-1)* 52 * 60 * 30 / 111111.1
            width =         1* 52 * 60 * 30 / 111111.1
            area = np.pi * (outer_r**2 - inner_r**2)
            alpha = min(0.8, size / max(area, 1))      # min and max keep it in [0,0.8] 
            
            if alpha > 0.01:
                if outer_r > 0:
                    # Draw base annulus
                    annulus = patches.Annulus((lon, lat), outer_r, width=width, 
                                          facecolor=facecolor, alpha=alpha, 
                                          edgecolor='none', transform=ccrs.PlateCarree())
                    ax.add_patch(annulus)


                    # Draw Coriolis spokes for this age 
                    lat_rad = np.radians(lat)
                    f = 2*2*np.pi/86400. *np.sin(lat_rad)  # Coriolis , f(in /s, lat)
                    
                    # Mean counterclockwise rotation increases with age for f>0
                    # If a ring of parcels contracts by a radial distance dr, 
                    # the area of the contraction annulus is set equal to size. 
                    # then vtan = f*dr, distance traversed is dxtan = f*dr*age, 
                    # dtheta = dxtan / (2*pi*r)
 
                    
                    for spoke_idx in range(n_spokes):
                        # Base azimuth for this spoke at outer_r 
                        base_azimuth = np.radians(spoke_idx * 360 / n_spokes)
                        
                        # Create curved spoke from outer_r inward to radius of mass sink (size)
                        n_points = 50
                        radii = np.linspace(outer_r, np.sqrt(size/np.pi), n_points) 

                        dr = size /2/np.pi/radii *111111.   # meters 
                        vtan = f*dr      # m/s, function of latitude
                        dxtan= vtan*age                     # meters 
                        dtheta = dxtan /2/np.pi/radii       # radians
                        
                        # Azimuth(radii), varies along spoke due to Coriolis
                        azimuths = base_azimuth + dtheta 
                        
                        # Convert to lon/lat offsets
                        x_offsets = radii * np.cos(azimuths)
                        y_offsets = radii * np.sin(azimuths)
                        
                        # Create curved path
                        spoke_lons = lon + x_offsets
                        spoke_lats = lat + y_offsets
                        
                        # Plot as curved line
                        ax.plot(spoke_lons, spoke_lats, color=spoke_color, \
                                transform=ccrs.PlateCarree(), \
                                alpha = alpha)   # min(0.9, 1 / max(area, 1)))   
                        
            else:
                to_remove.append(i)
        
        for i in reversed(to_remove):
            del self.brown_events[i]    

In [6]:
# WATERSHED get_feature_centroids_and_sizes, sizes in pixels (data_array, sigma=5, threshold=230, min_size=0)

from skimage.segmentation import watershed
from scipy.ndimage import distance_transform_edt
from scipy.ndimage import label, gaussian_filter

def get_feature_centroids_and_sizes(data_array, sigma=5, threshold=230, min_size=0):
    smimage = gaussian_filter(data_array.values, sigma=sigma)
    binary = smimage < threshold
    
    dist = distance_transform_edt(binary)
    labeled_array = watershed(-dist, mask=binary)
    
    lats = data_array.lat.values
    lons = data_array.lon.values
    
    centroids, sizes = [], []
    for fid in range(1, labeled_array.max() + 1):
        rows, cols = np.where(labeled_array == fid)
        size = len(rows)
        if size >= min_size:
            sizes.append(size)
            centroids.append((lats[int(rows.mean())], lons[int(cols.mean())]))
    
    return centroids, sizes  

#from matplotlib.patches import Circle  # for a graphical test 
#centroids,sizes = get_feature_centroids_and_sizes(data_array, sigma=5, threshold=230)
#data_array.plot()
#for (lat, lon), size in zip(centroids, sizes):
#    plt.gca().add_patch(Circle((lon, lat), radius=np.sqrt(size/200/np.pi), fill=False, edgecolor='red'))

# okay let's make the frames of animation 

In [7]:
datafiles

'/Users/bmapes/Box/Cupido_Aug2006/merg_2006081*_4km-pixel.nc4'

In [12]:
# Open data files 

from pathlib import Path
datafiles = '/Users/bmapes/Box/Cupido_Aug2006/' + \
            'merg_2006081*_4km-pixel.nc4' 
#            '/Users/bmapes/Box/Cupido_Aug2006/' + \
#            'merg_20060811*_4km-pixel.nc4'

            #'merg_2017081[3,4,5]*_4km-pixel.nc4' # some days 

OUTPUT = Path('/Users/bmapes/Downloads/Tucson_pinwheels2/')
OUTPUT.mkdir(exist_ok=True, parents=True) 
OUTDIR = str(OUTPUT)+'/'


ds = xr.open_mfdataset(datafiles)
Tb = ds.Tb[10:]  # [25:] means Start from 12Z on the first day for this dataset 
Tb

<xarray.DataArray 'Tb' (time: 86, lat: 632, lon: 688)> Size: 150MB
dask.array<getitem, shape=(86, 632, 688), dtype=float32, chunksize=(2, 632, 688), chunktype=numpy.ndarray>
Coordinates:
  * lon      (lon) float32 3kB -120.0 -120.0 -119.9 ... -95.07 -95.04 -95.0
  * lat      (lat) float32 3kB 22.03 22.07 22.1 22.14 ... 44.92 44.95 44.99
  * time     (time) datetime64[ns] 688B 2006-08-10T05:00:00.000013312 ... 200...
Attributes:
    units:          K
    standard_name:  brightness_temperature

In [16]:
# UPDATED create_masked_cmap function: a time dependent colormap with black masked range of Tb 
# time it is its parameter, retreating from the cold end except the orange is there too 

from matplotlib.colors import LinearSegmentedColormap, to_rgba
import numpy as np

def create_masked_cmap(it, vmin=190, vmax=340, verbose=False, 
                       mask_bot=220, mask_top=305, blackrate=1.0):
    """
    Create a colormap with a time-dependent retreating black mask.
    
    Args:
        it: time parameter (frame number or time index)
        vmin, vmax: data value range
        verbose: print mask range info
        mask_bot: bottom of mask band (retreats with time)
        mask_top: top of mask band (fixed)
        blackrate: how fast the mask retreats (Tb units per time step)
    
    Returns:
        cmap: LinearSegmentedColormap with masked band
    """
    
    # Define original colormap

    # Brigher colors from BlueMarble (less white haze at high T), less garish hot orange
    color_points = [
        (190, "#dc05ef", 1.0),  # magenta 
        (222, "#0589ef", 1.0),  # blue (color enhancement ends)
        (240, "#00ffff", 1.0),  # cyan
        (250, "#716f6f", 1.0),  # darker gray 
        (270, "#c5c6c6", 1.0),  # light-mid gray
        (280, "#ffffff", 0.7),  # white semitrans light gray
        (290, "#ffffff", 0.3),  # white point surface
        (300, "#ffffff", 0.1),  # SST point water
        (305, "#ffffff", 0.3),  # --> WHITE 
        (310, "#ff8000", 0.6),  # HOT surface orange, see https://html-color.codes/
        (340, "#000000", 0.8)  # HOTHOT surface black
    ]    
        
    # Create base colormap first
    cmap_colors = [((v - vmin) / (vmax - vmin), to_rgba(c, a)) for v, c, a in color_points]
    base_cmap = LinearSegmentedColormap.from_list("base_custom", cmap_colors)
    
    # Calculate mask band: [mask_bot + it*blackrate, mask_top]
    mask_lower = mask_bot + it * blackrate
    mask_upper = mask_top
    mask_lower_norm = (mask_lower - vmin) / (vmax - vmin)
    mask_upper_norm = (mask_upper - vmin) / (vmax - vmin)
    
    if verbose:
        print('mask range: ', mask_lower, mask_upper)
    
    # Sample the interpolated colormap densely, then mask it smoothly
    n_samples = 256
    pos_array = np.linspace(0, 1, n_samples)
    rgba_array = base_cmap(pos_array)
    
    # Set masked band to black with full opacity
    mask_idx = (pos_array >= mask_lower_norm) & (pos_array <= mask_upper_norm)
    rgba_array[mask_idx] = [0, 0, 0, 1.0]
    
    # Create new colormap from the masked array
    return LinearSegmentedColormap.from_list("masked_custom", 
                                             list(zip(pos_array, rgba_array)))

# Usage:
cmap = create_masked_cmap(it=10, blackrate=0, verbose=True)
# colored = cmap(normalized_data)

mask range:  220 305


In [17]:
# Read in big Blue Marble image 
# --- Place this code BEFORE your 1000-frame loop (Load ONCE) ---

# Disable the decompression bomb check
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

# --- Configuration (MUST match your image and desired georeference) ---
STATIC_MAP_FILE = '/Users/bmapes/Box/Sky_Symphony_Box/BlueMarble.200408.3x21600x10800.jpg' 
FULL_WORLD_EXTENT = [-180, 180, -90, 90] 
try:
    # Load the single satellite image file ONLY ONCE
    BASE_IMAGE_DATA = imread(STATIC_MAP_FILE)
except FileNotFoundError:
    raise FileNotFoundError(f"Error: Static map file '{STATIC_MAP_FILE}' not found. '+\
    'Please download a high-res Blue Marble image and save it to this name.")

In [ ]:
# Loop over times, making frames in the named folder frame_folder 

vmin = 190; vmax=340 # as in Irma movie frames
dpi = 150

# Initialize annotator, loading past state if this is a restart
# annotator = EyedropperAnnotator.load_state('annotator_state.pkl')
annotator = EyedropperAnnotator( )  

# Loop over the times, making the frames
for frame_count in range(len(ds.time)): 
    
    ######## Plotting code if file does not exist (for restart after kernel crashes) 
    out_file = OUTDIR + f"{frame_count:08d}.png"

# Grab Tb
    Tb = ds.Tb.isel(time = frame_count).interpolate_na(dim="lon", method="linear", max_gap=2)

# Convective Events 
    centroids,sizes = get_feature_centroids_and_sizes(Tb, sigma=7, threshold=230, 
                                                  min_size=10)  # pixels 
    sizes = np.array(sizes) * (4./111.)**2 # square degrees from 4km pixels
        
# Plot data, on top of blue marble image since there is some transparency 
    fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})
    
# Background image: cheap stock image? Naw, blue marble for August
    #ax.stock_img()  # low-res natural Earth background
    ax.imshow(BASE_IMAGE_DATA,      
      origin='upper', transform=ccrs.PlateCarree(), extent=FULL_WORLD_EXTENT)

# Add land and coastlines
    #ax.add_feature(cfeature.LAND)
    #ax.add_feature(cfeature.COASTLINE)

# Add state boundaries and coastlines 
    states_provinces = cfeature.NaturalEarthFeature(
        category='cultural', name='admin_1_states_provinces_lines',
        scale='50m', facecolor='none')
    ax.add_feature(states_provinces, edgecolor='white')
    
    ax.coastlines(color='white')
    
    ax.set_extent([ min(Tb.lon), max(Tb.lon), min(Tb.lat), max(Tb.lat) ], \
                  crs=ccrs.PlateCarree())
    ax.set_title(f"Tb at {Tb.time.values}", fontsize=12)

# image of the data   
    Tb.plot.imshow(
        ax=ax, transform=ccrs.PlateCarree(), cmap=cmap, vmin=vmin,vmax=vmax,\
        add_colorbar=False, #cbar_kwargs={'label': 'Brightness Temperature [K]'}
    )
    plt.tight_layout()
    
# Add eyedropper annotations    
    annotator.add_events(centroids, (np.array(sizes)).tolist(), frame_count)
# Brighter! Just boost the size values
#    annotator.add_events(centroids, (2*np.array(sizes)).tolist(), frame_count)

    #print('centroids ', centroids)
    #print('sizes ', sizes)

#    annotator.plot_one_expanding_ring(ax, frame_count, facecolor='orange')
#    annotator.plot_expanding_ring_coriolis(ax, frame_count, 
    annotator.plot_expanding_ring_coriolis_pinwheel(ax, frame_count, 
                                   facecolor='orange', 
                                   spoke_color='white', 
                                   n_spokes=8)
   
# Cover annotations with zorder=10, near the souce where they are too hokey 
#    Tb.where(Tb < 230).plot.imshow(ax=ax, add_colorbar=False, cmap=cmap, vmin=vmin,vmax=vmax, zorder=10)
    
    plt.savefig(out_file, dpi=dpi)
    plt.close()
    print(f"Saved {out_file}")
    print(f"Tb at {Tb.time.values}")


Saved /Users/bmapes/Downloads/Tucson_pinwheels2/00000000.png
Tb at 2006-08-10T00:00:00.000000000
Saved /Users/bmapes/Downloads/Tucson_pinwheels2/00000001.png
Tb at 2006-08-10T00:30:00.000013312
